In [1]:
import sys
import os

In [ ]:
sys.path.append(os.path.abspath('../../../../src'))

from read_and_write_docs import read_jsonl, read_rds
from tokenize_and_score import load_model
from utils import apply_temp_doc_id, build_metadata_df
from n_gram_functions import (
    common_ngrams,
    pretty_print_common_ngrams,
    keep_before_phrase,
    score_phrases,
    add_pmf_column,
    score_phrases_no_context
)

In [3]:
corpus = "Wiki"
data_type = "training"

known_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/known_raw.jsonl"
known = read_jsonl(known_loc)
known = apply_temp_doc_id(known)

unknown_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/{corpus}/unknown_raw.jsonl"
unknown = read_jsonl(unknown_loc)
unknown_df = apply_temp_doc_id(unknown)

metadata_loc = f"/Volumes/BCross/datasets/author_verification/{data_type}/metadata.rds"
metadata = read_rds(metadata_loc)
filtered_metadata = metadata[metadata['corpus'] == corpus]
agg_metadata = build_metadata_df(filtered_metadata, known, unknown)

In [4]:
problem_dataset_agg = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_agg.jsonl")
problem_dataset_profile = read_jsonl("/Users/user/Documents/test_data/n-gram_tracing/Wiki_training_profile.jsonl")

In [5]:
same_author_problems = problem_dataset_agg[problem_dataset_agg['known_author'] == problem_dataset_agg['unknown_author']].copy()
same_author_problems.sort_values(["highest_common_count"], ascending=[False], inplace=True)
same_author_problems[(same_author_problems['highest_common_count'] >= 3) & (same_author_problems['highest_common_count'] <= 10)]

,problem,known_author,unknown_author,known_doc_id,unknown_doc_id,highest_common_count,highest_common_ngram
379,Greg_L vs Greg_L,Greg_L,Greg_L,greg_l_text_11,greg_l_text_10,9,", Ġthey Ġshould Ġhave Ġparticipated Ġin Ġthe Ġ..."
432,Haymaker vs Haymaker,Haymaker,Haymaker,haymaker_text_3,haymaker_text_2,9,"Ġat Ġthe Ġend Ġof Ġthe Ġday , Ġwe 're"
354,Fragments_of_Jade vs Fragments_of_Jade,Fragments_of_Jade,Fragments_of_Jade,fragments_of_jade_text_2,fragments_of_jade_text_10,8,"Ġme , Ġand Ġit 's Ġgetting Ġold .Ċ"
337,Fixentries vs Fixentries,Fixentries,Fixentries,fixentries_text_2,fixentries_text_5,8,Ġthe Ġindividual Ġher it ability Ġof Ġintellig...
248,DonaNobisPacem vs DonaNobisPacem,DonaNobisPacem,DonaNobisPacem,donanobispacem_text_5,donanobispacem_text_2,8,Ġafter Ġ 1 8 - 2 0 Ġweeks
...,...,...,...,...,...,...,...
222,David_Shankbone vs David_Shankbone,David_Shankbone,David_Shankbone,david_shankbone_text_1,david_shankbone_text_4,3,", Ġwhich Ġis"
211,D7G1DX~0 vs D7G1DX~0,D7G1DX~0,D7G1DX~0,d7g1dx_0_text_2,d7g1dx_0_text_5,3,Ġdon 't Ġthink
187,Collect vs Collect,Collect,Collect,collect_text_12,collect_text_11,3,", Ġand Ġi"
198,Cptnono vs Cptnono,Cptnono,Cptnono,cptnono_text_1,cptnono_text_12,3,", Ġthough .Ċ"


In [6]:
known_doc = "greg_l_text_11"
known_text = known[known['doc_id'] == known_doc].reset_index().loc[0, 'text']

unknown_doc = "greg_l_text_10"
unknown_text = unknown[unknown['doc_id'] == unknown_doc].reset_index().loc[0, 'text']

In [7]:
tokenizer, model = load_model("/Volumes/BCross/models/Qwen 2.5/Qwen2.5-0.5B-Instruct")

In [39]:
common = common_ngrams(known_text, unknown_text, 2, model, tokenizer, lowercase=True)

In [40]:
pretty_print_common_ngrams(common, tokenizer=tokenizer, order='len_asc')

2-grams (5): [' consensus view', ' of the', ' such as', ' was a', '.\nyou']
3-grams (3): [' dicklyon', ' meritorious', ' the eight editors']
4-grams (1): [' had felt they had']
9-grams (2): [' allege harbor views that are contrary to the consensus', ', they should have participated in the poll.\n']


In [10]:
print(f"""
<DOC>
{known_text}
</DOC>
<NGRAM>
""")


<DOC>
We simply can not permit further stonewalling via tactics such as avoiding high-profile efforts at If the eight editors you point to and allege harbor views that are contrary to the consensus truly had felt they had a meritorious argument, they should have participated in the poll.
We simply can not permit further stonewalling via tactics such avoiding high-profile efforts at Your argument is nonsense and merits no further response than this The wording is not irrelevant to the 17 people who participated in the poll.
The consensus view is against your reasoning, as well as your wishes.
You could have voted this, that, or something else.
Your argument is nonsense and merits no further response than this You could have voted this, that, or something else.
Try reading Wikipedia s Whether or not you agree with the 17 people who participated in the poll and the new numbers reinforcing that message in Dicklyon s, you will have to respect it.
TheFreeloader, there was a minority crew of

## Phrase 1 -  " allege harbor views that are contrary to the consensus"

In [11]:
p_1 = " allege harbor views that are contrary to the consensus"

para_1 = [ " allege that they harbor views that are contrary to the consensus", " allege that they harbor views that are opposed to the consensus", " allege that they harbor views that are at odds with the consensus", " allege that they harbor opinions that are contrary to the consensus", " allege that they harbor opinions that are opposed to the consensus", " allege that they harbor opinions that are at odds with the consensus", " allege that they hold views that are contrary to the consensus", " allege that they hold views that are opposed to the consensus", " allege that they hold views that are at odds with the consensus", " allege that they hold opinions that are contrary to the consensus", " allege that they hold opinions that are opposed to the consensus", " allege that they hold opinions that are at odds with the consensus", " allege that they entertain views that are contrary to the consensus", " allege that they entertain views that are opposed to the consensus", " allege that they entertain views that are at odds with the consensus", " allege that they entertain opinions that are contrary to the consensus", " allege that they entertain opinions that are opposed to the consensus", " allege that they entertain opinions that are at odds with the consensus", " claim that they harbor views that are contrary to the consensus", " claim that they harbor views that are opposed to the consensus", " claim that they harbor views that are at odds with the consensus", " claim that they harbor opinions that are contrary to the consensus", " claim that they harbor opinions that are opposed to the consensus", " claim that they harbor opinions that are at odds with the consensus", " claim that they hold views that are contrary to the consensus", " claim that they hold views that are opposed to the consensus", " claim that they hold views that are at odds with the consensus", " claim that they hold opinions that are contrary to the consensus", " claim that they hold opinions that are opposed to the consensus", " claim that they hold opinions that are at odds with the consensus", " claim that they entertain views that are contrary to the consensus", " claim that they entertain views that are opposed to the consensus", " claim that they entertain views that are at odds with the consensus", " claim that they entertain opinions that are contrary to the consensus", " claim that they entertain opinions that are opposed to the consensus", " claim that they entertain opinions that are at odds with the consensus", " assert that they harbor views that are contrary to the consensus", " assert that they harbor views that are opposed to the consensus", " assert that they harbor views that are at odds with the consensus", " assert that they harbor opinions that are contrary to the consensus", " assert that they harbor opinions that are opposed to the consensus", " assert that they harbor opinions that are at odds with the consensus", " assert that they hold views that are contrary to the consensus", " assert that they hold views that are opposed to the consensus", " assert that they hold views that are at odds with the consensus", " assert that they hold opinions that are contrary to the consensus", " assert that they hold opinions that are opposed to the consensus", " assert that they hold opinions that are at odds with the consensus", " assert that they entertain views that are contrary to the consensus", " assert that they entertain views that are opposed to the consensus", " assert that they entertain views that are at odds with the consensus", " assert that they entertain opinions that are contrary to the consensus", " assert that they entertain opinions that are opposed to the consensus", " assert that they entertain opinions that are at odds with the consensus", " contend that they harbor views that are contrary to the consensus", " contend that they harbor views that are opposed to the consensus", " contend that they harbor views that are at odds with the consensus", " contend that they harbor opinions that are contrary to the consensus", " contend that they harbor opinions that are opposed to the consensus", " contend that they harbor opinions that are at odds with the consensus", " contend that they hold views that are contrary to the consensus", " contend that they hold views that are opposed to the consensus", " contend that they hold views that are at odds with the consensus", " contend that they hold opinions that are contrary to the consensus", " contend that they hold opinions that are opposed to the consensus", " contend that they hold opinions that are at odds with the consensus", " contend that they entertain views that are contrary to the consensus", " contend that they entertain views that are opposed to the consensus", " contend that they entertain views that are at odds with the consensus", " contend that they entertain opinions that are contrary to the consensus", " contend that they entertain opinions that are opposed to the consensus", " contend that they entertain opinions that are at odds with the consensus", " allege that they harbour views that are contrary to the consensus", " allege that they harbour views that are opposed to the consensus", " allege that they harbour views that are at odds with the consensus", " allege that they harbour opinions that are contrary to the consensus", " allege that they harbour opinions that are opposed to the consensus", " allege that they harbour opinions that are at odds with the consensus", " assert that they harbour views that are contrary to the consensus", " assert that they harbour views that are opposed to the consensus", " assert that they harbour views that are at odds with the consensus", " assert that they harbour opinions that are contrary to the consensus", " assert that they harbour opinions that are opposed to the consensus", " assert that they harbour opinions that are at odds with the consensus", " allege that they have views that are contrary to the consensus", " allege that they have views that are at odds with the consensus", " claim that they have views that are contrary to the consensus", " claim that they have views that are at odds with the consensus", " assert that they have views that are contrary to the consensus", " assert that they have views that are at odds with the consensus", " allege that they hold positions that are contrary to the consensus", " allege that they hold positions that are opposed to the consensus", " allege that they hold positions that are at odds with the consensus", " claim that they hold positions that are contrary to the consensus", " claim that they hold positions that are opposed to the consensus", " claim that they hold positions that are at odds with the consensus", " allege that they harbor views that are against the consensus", " allege that they harbor views that are in conflict with the consensus", " allege that they harbor views that are inconsistent with the consensus", " allege that they harbor views that are counter to the consensus", " make the allegation that they hold views that are contrary to the consensus", " make the allegation that they hold views that are opposed to the consensus", " make the allegation that they hold views that are at odds with the consensus", " make the claim that they hold views that are contrary to the consensus", " make the claim that they hold views that are opposed to the consensus", " make the claim that they hold views that are at odds with the consensus", " make the assertion that they hold views that are contrary to the consensus", " make the assertion that they hold views that are opposed to the consensus", " make the assertion that they hold views that are at odds with the consensus", " maintain that they harbor beliefs that are contrary to the consensus", " maintain that they harbor beliefs that are opposed to the consensus", " maintain that they harbor beliefs that are at odds with the consensus", " aver that they harbor beliefs that are contrary to the consensus", " aver that they harbor beliefs that are opposed to the consensus", " aver that they harbor beliefs that are at odds with the consensus", " allege they harbor views that are contrary to the consensus", " allege they harbor views that are opposed to the consensus", " allege they harbor views that are at odds with the consensus", " claim they harbor views that are contrary to the consensus", " claim they harbor views that are opposed to the consensus", " claim they harbor views that are at odds with the consensus", " assert they harbor views that are contrary to the consensus", " assert they harbor views that are opposed to the consensus", " assert they harbor views that are at odds with the consensus", " contend they hold opinions that are contrary to the consensus", " contend they hold opinions that are at odds with the consensus", " assert that they entertain beliefs that are contrary to the consensus", " assert that they entertain beliefs that are at odds with the consensus", " allege that they harbour beliefs that are contrary to the consensus", " allege that they harbour beliefs that are at odds with the consensus", " maintain they hold views that are contrary to the consensus", " maintain they hold views that are at odds with the consensus", " aver they hold views that are contrary to the consensus", " aver they hold views that are at odds with the consensus", " claim that they harbour opinions that are against the consensus" ]

known_base_p1 = keep_before_phrase(known_text, p_1, True)
unknown_base_p1 = keep_before_phrase(unknown_text, p_1, True)

In [12]:
known_p1_scores = score_phrases(known_base_p1, p_1, para_1, tokenizer, model)
unknown_p1_scores = score_phrases(unknown_base_p1, p_1, para_1, tokenizer, model)

→ Scoring base_text alone…
   base_total = -167.7171

→ [1/136] Processing reference…
→ [2/136] Processing paraphrase…
→ [3/136] Processing paraphrase…
→ [4/136] Processing paraphrase…
→ [5/136] Processing paraphrase…
→ [6/136] Processing paraphrase…
→ [7/136] Processing paraphrase…
→ [8/136] Processing paraphrase…
→ [9/136] Processing paraphrase…
→ [10/136] Processing paraphrase…
→ [11/136] Processing paraphrase…
→ [12/136] Processing paraphrase…
→ [13/136] Processing paraphrase…
→ [14/136] Processing paraphrase…
→ [15/136] Processing paraphrase…
→ [16/136] Processing paraphrase…
→ [17/136] Processing paraphrase…
→ [18/136] Processing paraphrase…
→ [19/136] Processing paraphrase…
→ [20/136] Processing paraphrase…
→ [21/136] Processing paraphrase…
→ [22/136] Processing paraphrase…
→ [23/136] Processing paraphrase…
→ [24/136] Processing paraphrase…
→ [25/136] Processing paraphrase…
→ [26/136] Processing paraphrase…
→ [27/136] Processing paraphrase…
→ [28/136] Processing paraphrase…
→ [2

In [13]:
known_p1_pmf = add_pmf_column(known_p1_scores, 'phrase_log_probs')
unknown_p1_pmf = add_pmf_column(unknown_p1_scores, 'phrase_log_probs')

In [14]:
p1_pmf = score_phrases_no_context(p_1, para_1, tokenizer, model)
p1_pmf = add_pmf_column(p1_pmf, 'log_probs')

→ [1/136] Processing reference…
→ [2/136] Processing paraphrase…
→ [3/136] Processing paraphrase…
→ [4/136] Processing paraphrase…
→ [5/136] Processing paraphrase…
→ [6/136] Processing paraphrase…
→ [7/136] Processing paraphrase…
→ [8/136] Processing paraphrase…
→ [9/136] Processing paraphrase…
→ [10/136] Processing paraphrase…
→ [11/136] Processing paraphrase…
→ [12/136] Processing paraphrase…
→ [13/136] Processing paraphrase…
→ [14/136] Processing paraphrase…
→ [15/136] Processing paraphrase…
→ [16/136] Processing paraphrase…
→ [17/136] Processing paraphrase…
→ [18/136] Processing paraphrase…
→ [19/136] Processing paraphrase…
→ [20/136] Processing paraphrase…
→ [21/136] Processing paraphrase…
→ [22/136] Processing paraphrase…
→ [23/136] Processing paraphrase…
→ [24/136] Processing paraphrase…
→ [25/136] Processing paraphrase…
→ [26/136] Processing paraphrase…
→ [27/136] Processing paraphrase…
→ [28/136] Processing paraphrase…
→ [29/136] Processing paraphrase…
→ [30/136] Processing pa

## Phrase 2 - ", they should have participated in the poll.\n"

In [15]:
p_2 = ", they should have participated in the poll.\n"

para_2 = [", they should've participated in the poll.\n", ", they should have taken part in the poll.\n", ", they should've taken part in the poll.\n", ", they should have joined in the poll.\n", ", they should've joined in the poll.\n", ", they should have joined the poll.\n", ", they should've joined the poll.\n", ", they should have been part of the poll.\n", ", they should've been part of the poll.\n", ", they should have taken part in the polling.\n", ", they should've taken part in the polling.\n", ", they should have participated in the polling.\n", ", they should've participated in the polling.\n", ", they should have engaged in the poll.\n", ", they should've engaged in the poll.\n", ", they should have partaken in the poll.\n", ", they should've partaken in the poll.\n", ", in the poll they should have participated.\n", ", in the poll they should've participated.\n", ", the poll should have been participated in by them.\n", ", the poll should've been participated in by them.\n"]

known_base_p2 = keep_before_phrase(known_text, p_2, True)
unknown_base_p2 = keep_before_phrase(unknown_text, p_2, True)

In [16]:
known_p2_scores = score_phrases(known_base_p2, p_2, para_2, tokenizer, model)
unknown_p2_scores = score_phrases(unknown_base_p2, p_2, para_2, tokenizer, model)

→ Scoring base_text alone…
   base_total = -261.7848

→ [1/22] Processing reference…
→ [2/22] Processing paraphrase…
→ [3/22] Processing paraphrase…
→ [4/22] Processing paraphrase…
→ [5/22] Processing paraphrase…
→ [6/22] Processing paraphrase…
→ [7/22] Processing paraphrase…
→ [8/22] Processing paraphrase…
→ [9/22] Processing paraphrase…
→ [10/22] Processing paraphrase…
→ [11/22] Processing paraphrase…
→ [12/22] Processing paraphrase…
→ [13/22] Processing paraphrase…
→ [14/22] Processing paraphrase…
→ [15/22] Processing paraphrase…
→ [16/22] Processing paraphrase…
→ [17/22] Processing paraphrase…
→ [18/22] Processing paraphrase…
→ [19/22] Processing paraphrase…
→ [20/22] Processing paraphrase…
→ [21/22] Processing paraphrase…
→ [22/22] Processing paraphrase…
→ Scoring base_text alone…
   base_total = -2955.7817

→ [1/22] Processing reference…
→ [2/22] Processing paraphrase…
→ [3/22] Processing paraphrase…
→ [4/22] Processing paraphrase…
→ [5/22] Processing paraphrase…
→ [6/22] Process

In [17]:
known_p2_pmf = add_pmf_column(known_p2_scores, 'phrase_log_probs')
unknown_p2_pmf = add_pmf_column(unknown_p2_scores, 'phrase_log_probs')

In [18]:
p2_pmf = score_phrases_no_context(p_2, para_2, tokenizer, model)
p2_pmf = add_pmf_column(p2_pmf, 'log_probs')

→ [1/22] Processing reference…
→ [2/22] Processing paraphrase…
→ [3/22] Processing paraphrase…
→ [4/22] Processing paraphrase…
→ [5/22] Processing paraphrase…
→ [6/22] Processing paraphrase…
→ [7/22] Processing paraphrase…
→ [8/22] Processing paraphrase…
→ [9/22] Processing paraphrase…
→ [10/22] Processing paraphrase…
→ [11/22] Processing paraphrase…
→ [12/22] Processing paraphrase…
→ [13/22] Processing paraphrase…
→ [14/22] Processing paraphrase…
→ [15/22] Processing paraphrase…
→ [16/22] Processing paraphrase…
→ [17/22] Processing paraphrase…
→ [18/22] Processing paraphrase…
→ [19/22] Processing paraphrase…
→ [20/22] Processing paraphrase…
→ [21/22] Processing paraphrase…
→ [22/22] Processing paraphrase…


## Phrase 3 - " had felt they had"

In [19]:
p_3 = " had felt they had"

para_3 = [" had felt they'd", " had felt that they had", " had felt that they'd", " had believed they had", " had believed they'd", " had believed that they had", " had believed that they'd", " had thought they had", " had thought they'd", " had thought that they had", " had thought that they'd", " had supposed they had", " had supposed they'd", " had supposed that they had", " had supposed that they'd", " had reckoned they had", " had reckoned they'd", " had reckoned that they had", " had reckoned that they'd", " had sensed they had", " had sensed they'd", " had sensed that they had", " had sensed that they'd", " had perceived they had", " had perceived they'd", " had perceived that they had", " had perceived that they'd", " had suspected they had", " had suspected they'd", " had suspected that they had", " had suspected that they'd", " had assumed they had", " had assumed they'd", " had assumed that they had", " had assumed that they'd", " had had a feeling they had", " had had a feeling they'd", " had had the feeling they had", " had had the feeling they'd", " had felt they possessed", " had felt that they possessed", " had believed they possessed", " had thought they possessed", " had supposed they possessed", " had reckoned they possessed", " had sensed they possessed", " had perceived they possessed", " had suspected they possessed", " had assumed they possessed", " had had a sense they had", " had had the sense they had", " had had a sense that they had", " had had the sense that they had", " had opined they had", " had opined that they had", " had concluded they had", " had concluded that they had"]

known_base_p3 = keep_before_phrase(known_text, p_3, True)
unknown_base_p3 = keep_before_phrase(unknown_text, p_3, True)

In [20]:
known_p3_scores = score_phrases(known_base_p3, p_3, para_3, tokenizer, model)
unknown_p3_scores = score_phrases(unknown_base_p3, p_3, para_3, tokenizer, model)

→ Scoring base_text alone…
   base_total = -227.1298

→ [1/58] Processing reference…
→ [2/58] Processing paraphrase…
→ [3/58] Processing paraphrase…
→ [4/58] Processing paraphrase…
→ [5/58] Processing paraphrase…
→ [6/58] Processing paraphrase…
→ [7/58] Processing paraphrase…
→ [8/58] Processing paraphrase…
→ [9/58] Processing paraphrase…
→ [10/58] Processing paraphrase…
→ [11/58] Processing paraphrase…
→ [12/58] Processing paraphrase…
→ [13/58] Processing paraphrase…
→ [14/58] Processing paraphrase…
→ [15/58] Processing paraphrase…
→ [16/58] Processing paraphrase…
→ [17/58] Processing paraphrase…
→ [18/58] Processing paraphrase…
→ [19/58] Processing paraphrase…
→ [20/58] Processing paraphrase…
→ [21/58] Processing paraphrase…
→ [22/58] Processing paraphrase…
→ [23/58] Processing paraphrase…
→ [24/58] Processing paraphrase…
→ [25/58] Processing paraphrase…
→ [26/58] Processing paraphrase…
→ [27/58] Processing paraphrase…
→ [28/58] Processing paraphrase…
→ [29/58] Processing paraphrase…

In [21]:
known_p3_pmf = add_pmf_column(known_p3_scores, 'phrase_log_probs')
unknown_p3_pmf = add_pmf_column(unknown_p3_scores, 'phrase_log_probs')

In [33]:
unknown_p3_pmf.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
p3_pmf = score_phrases_no_context(p_3, para_3, tokenizer, model)
p3_pmf = add_pmf_column(p3_pmf, 'log_probs')

→ [1/58] Processing reference…
→ [2/58] Processing paraphrase…
→ [3/58] Processing paraphrase…
→ [4/58] Processing paraphrase…
→ [5/58] Processing paraphrase…
→ [6/58] Processing paraphrase…
→ [7/58] Processing paraphrase…
→ [8/58] Processing paraphrase…
→ [9/58] Processing paraphrase…
→ [10/58] Processing paraphrase…
→ [11/58] Processing paraphrase…
→ [12/58] Processing paraphrase…
→ [13/58] Processing paraphrase…
→ [14/58] Processing paraphrase…
→ [15/58] Processing paraphrase…
→ [16/58] Processing paraphrase…
→ [17/58] Processing paraphrase…
→ [18/58] Processing paraphrase…
→ [19/58] Processing paraphrase…
→ [20/58] Processing paraphrase…
→ [21/58] Processing paraphrase…
→ [22/58] Processing paraphrase…
→ [23/58] Processing paraphrase…
→ [24/58] Processing paraphrase…
→ [25/58] Processing paraphrase…
→ [26/58] Processing paraphrase…
→ [27/58] Processing paraphrase…
→ [28/58] Processing paraphrase…
→ [29/58] Processing paraphrase…
→ [30/58] Processing paraphrase…
→ [31/58] Processing